<a href="https://colab.research.google.com/github/JayanthMalladi/hugging-face-transformers-LLM/blob/main/fullTrainingLoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install "transformers[sentencepiece]"
!pip install torch
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from datasets import load_dataset

raw_dataset = load_dataset("glue", "mrpc")

checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
tokenized_dataset=tokenized_dataset.remove_columns(['idx'])


In [10]:
tokenized_dataset=tokenized_dataset.remove_columns(['sentence1' , 'sentence2'])

In [5]:
tokenized_dataset = tokenized_dataset.rename_column("label" , 'labels')

In [6]:
tokenized_dataset.set_format("torch")

In [7]:
tokenized_dataset["train"]

Dataset({
    features: ['sentence1', 'sentence2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

In [13]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle = True , batch_size = 8, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    tokenized_dataset["validation"], batch_size = 8, collate_fn = data_collator
)


In [14]:
for batch in train_dataloader:
      break
{k: v.shape for k, v in batch.items()}


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 62]),
 'token_type_ids': torch.Size([8, 62]),
 'attention_mask': torch.Size([8, 62])}

In [22]:
for k , v in batch.items():
  print(v.to(device))

tensor([0, 1, 1, 1, 1, 0, 1, 1], device='cuda:0')
tensor([[  101,  3960,  8040, 10606, 21222,  1024,  2008,  1005,  1055,  6149,
          1010,  1998,  1996,  5656,  2369,  2008,  2003,  1010,  2057,  2359,
          2000,  2785,  1997,  5610,  2032,  2041,  1010,  2000,  2156,  2073,
          2002,  2001,  2746,  2013,  1012,   102,  1000,  2057,  2359,  2000,
          2785,  1997,  5610,  2032,  2041,  1010,  2000,  2156,  2073,  2002,
          2001,  2746,  2013,  1010,  1000,  2056,  2728,  8040, 10606, 21222,
          1012,   102],
        [  101,  1000,  4415,  2009,  2003,  1037, 13800,  2154,  2005,  4841,
          1010,  1000,  2002,  2056,  1999,  2899,  1012,   102,  1000,  2009,
          1005,  1055,  4415,  1037, 13800,  2154,  2005,  2637,  1010,  1000,
          3639,  3187,  6221,  1044,  1012, 19379, 22747, 14273,  2056,  1999,
          2899,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,

In [15]:
outputs = model(**batch)

In [16]:
outputs.loss

tensor(0.6637, grad_fn=<NllLossBackward0>)

In [17]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters() , lr = 5e-5)

In [18]:
from transformers import get_scheduler

num_epochs = 3
training_steps  = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
  "linear",
  optimizer = optimizer,
  num_warmup_steps = 0,
  num_training_steps = training_steps
)


In [20]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [24]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [26]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [27]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8504901960784313, 'f1': 0.8964346349745331}